In [2]:
# nbi:hide_in
from ipywidgets import interact
def square(x):
    return x * x

interact(square, x=10);

interactive(children=(IntSlider(value=10, description='x', max=30, min=-10), Output()), _dom_classes=('widget-…